In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime
import pytz

In [2]:
db_user = "postgres"
db_password = "tranbaotin"
db_host = "localhost"
db_port = "5432"
db_name = "postgres"

In [3]:
db_connection_str = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Đường dẫn đến file CSV
csv_file_path = "etl_job.csv"

In [4]:
try:
    # Tạo engine kết nối đến database
    engine = create_engine(db_connection_str)
    
    # Đọc file CSV vào DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Điền các giá trị trống (NaN) bằng NaT
    df['START_TS'] = df['START_TS'].replace(np.nan, pd.NaT)
    df['END_TS'] = df['END_TS'].replace(np.nan, pd.NaT)
    
    current_time = datetime.now(pytz.timezone('Asia/Bangkok'))
    df['LAST_UPDATED_TS'] = current_time
    
    # THAY ĐỔI TÊN CỘT THÀNH CHỮ THƯỜNG
    df.columns = [col.lower() for col in df.columns]
    
    # Ghi DataFrame vào bảng etl.etl_job
    df.to_sql(
        name="etl_job",  # Tên bảng
        schema="etl",    # Schema
        con=engine,
        if_exists="append",  # Thêm dữ liệu vào bảng, không ghi đè
        index=False  # Không ghi cột index của DataFrame
    )
    
    print("Dữ liệu từ file CSV đã được import thành công vào bảng etl.etl_job")

except FileNotFoundError:
    print(f"Không tìm thấy file CSV tại: {csv_file_path}")
except ValueError as ve:
    print(f"Lỗi: {ve}")
except SQLAlchemyError as e:
    print(f"Lỗi khi import dữ liệu vào database: {e}")
finally:
    # Đóng kết nối nếu engine đã được tạo
    if 'engine' in locals():
        engine.dispose()

Dữ liệu từ file CSV đã được import thành công vào bảng etl.etl_job
